In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import warnings
import random
pd.options.display.float_format = '{:.2f}'.format
warnings.simplefilter(action='ignore')
sn.set()

In [2]:
data = pd.read_csv("housing_data.csv")

### Variables types

In [3]:
cols = [var for var in data.columns if var != "MLS"]
num_cols = [var for var in cols if data[var].dtype != 'O' and var != 'year_built']
cat_cols = [var for var in cols if data[var].dtype == 'O']
date_cols = [var for var in cols if var not in num_cols + cat_cols]

### Change "None" values to NaN

In [4]:
def fillingNones(df,cols):
    
    '''This function looks for all None string values and changes them to nan'''
    df = df.copy()
    
    #Iterates over the columns classified as "objects", containing strings
    for var in cols:
        df[var] = np.where(df[var] == "None", np.nan, df[var])
    return df

In [5]:
data = fillingNones(data,cat_cols)

### Dropping values from date feature

In [6]:
data = data[(data["year_built"] != 0) & (data["year_built"] > 1921)].reset_index(drop = True)

### Dropping zipcodes containing less than 6 observations

In [7]:
codes = (data.groupby("zipcode")["sold_price"].count().sort_values(ascending = False))
codes = codes.loc[codes > 5].index.tolist()
data = data.loc[data["zipcode"].isin(codes)].reset_index(drop = True)

### Dropping outliers from bedrooms

In [8]:
data = data.loc[data["bedrooms"] < 20]

### Dropping high value houses 

In [9]:
data = data.loc[data["sold_price"] < 1500000]

### Missing values

In [10]:
missing = [var for var in data.columns if data[var].isnull().sum() > 0]

### Filling NaN

In [11]:
for var in missing:

    # calculate the mode
    mode_val = data[var].mode()[0]

    # replace missing values by the mode
    data[var] = data[var].fillna(mode_val)

# check that we have no more missing values in the engineered variables
data[missing].isnull().sum()

lot_acres           0
bathrooms           0
sqrt_ft             0
garage              0
kitchen_features    0
fireplaces          0
floor_covering      0
HOA                 0
dtype: int64

### Cleaning values in HOA feature

In [12]:
data["HOA"] = data["HOA"].replace({"20,000":20000, "2,000":2000, "1,290": 1290, "1,769":1769,
                                   "8,333": 8333, "1,000":1000, "19,480":19480, '1,600':1600, 
                                   "1,010": 1010, "1,200":1200, "1,717": 1717, "1,100":1100,
                                  "1,270":1270, "5,900":5900})

In [13]:
data["HOA"] = data["HOA"].astype(float).astype(int)

### Casting type variables

In [14]:
integers = ["MLS","sold_price","zipcode","year_built","bedrooms","fireplaces"]
decimals = ["bathrooms","sqrt_ft","garage","fireplaces"]

In [15]:
def castObjectVars(df,cols_int,cols_float):
    '''This function casts dtype objects to a better fit'''
    
    df = df.copy()
    for x in cols_int:
        df[x] = data[x].astype(int)
    
    for i in cols_float:
        df[i] = data[i].values.astype(float).tolist()
    return df

In [16]:
data = castObjectVars(data,integers,decimals)

### Update features

In [17]:
cols = [var for var in data.columns if var != "MLS"]
num_cols = [var for var in cols if data[var].dtype != 'O' and var != 'year_built']
cat_cols = [var for var in cols if data[var].dtype == 'O']
date_cols = [var for var in cols if var not in num_cols + cat_cols]

discrete = [var for var in num_cols if len(data[var].unique()) < 22 and var not in date_cols+["MLS"]]
continuous = [var for var in num_cols if var not in discrete+date_cols+["MLS"]]

### Transform year built to house age

In [18]:
data["house_age"] = 2022 - data["year_built"]
data = data.drop(["year_built"], axis = 1).reset_index(drop = True)

### Create house initial values and some adding noise 

In [19]:
data["initial_price"] = data["sold_price"]/(pow(1 + random.uniform(0.04, 0.06) , data["house_age"])
                                            *random.uniform(1.5, 2.5))

data["soldprice_2032"] = data["sold_price"] * (pow(1 + 0.05 ,10)*random.uniform(1.7, 1.9))

In [20]:
data = castObjectVars(data,["initial_price","soldprice_2032"],decimals)

### Numerical variables transformation

In [21]:
for var in ["sold_price", "sqrt_ft", "initial_price", "soldprice_2032"]:    
    data[var] = np.log(data[var])

### Encoding kitchen features and floor covering

In [22]:
""" For the kitchen_features feature, it is necessary to first separate the
characteristics into different columns. After that, we can observe which ones
each house contains, and then create a new column with the vectorized
representations of them (not one-hot encoders). """

# Join every single characteristic into one list
feature = data["kitchen_features"]
kitchen_chars = []
kitchen_vectors = []

for i in feature:
    description = i.lower().split(",")
    kitchen_chars += description

# Create a vector with all the unique values
labels = []

for i in kitchen_chars:
    if ':' in i:
        aux = i.split(':')[0]
        labels.append(aux) #Just keep the general feature
    #print(f"Saving {aux}")
    elif i[0] == ' ':
        aux = i[1:]
        labels.append(aux)
    else:
        labels.append(i)
    #print(f"Saving {i}")

labels = np.unique(labels) 

#These labels should be given to the programming team as additional data
labels = ['appliance_color','countertops','microwave','electric_oven','convection_oven','gas_oven','compactor','desk',
          'dishwasher','sink','freezer','garbage_disposal','pantry','indoor_grill','lazy_susan','refrigerator',
          'water_purifier','warming_drawer','wet_bar','wine_cooler']

for i in feature:
    encoder = np.zeros(20)
    description = i.lower().split(",")

    for j in description:
        if ':' in j:
            j = j.split(':')[0]
    
        j = j.replace(' ','_')
    
    for k in range(len(labels)):
        if labels[k] in j:
            encoder[k] = 1
        else:
            if j == 'black' or j == 'stainless':
                encoder[0] = 1
            if j == 'butch_block' or j == 'island' or j == 'quartzite':
                encoder[1] = 1
            if 'electric' in j:
                encoder[3] = 1
            if j == 'oven' or 'convection' in j or j == '#_of_ovens':
                encoder[4] = 1
            if 'gas' in j:
                encoder[5] = 1
            if 'osmosis' in j:
                encoder[16] = 1
  
    kitchen_vectors.append(encoder)

# Create the columns with the presence of each feature
appliance = []
countertop = []
microwave = []
electric = []
convection = []
gas = []
compactor = []
desk = []
dishwasher = []
sink = []
freezer = []
garbage = []
pantry = []
grill = []
lazy = []
refrigerator = []
purifier = []
warming = []
wet = []
wine = []

for i in kitchen_vectors:
    appliance.append(i[0])
    countertop.append(i[1])
    microwave.append(i[2])
    electric.append(i[3])
    convection.append(i[4])
    gas.append(i[5])
    compactor.append(i[6])
    desk.append(i[7])
    dishwasher.append(i[8])
    sink.append(i[9])
    freezer.append(i[10])
    garbage.append(i[11])
    pantry.append(i[12])
    grill.append(i[13])
    lazy.append(i[14])
    refrigerator.append(i[15])
    purifier.append(i[16])
    warming.append(i[17])
    wet.append(i[18])
    wine.append(i[19])

data['appliance'] = appliance
data['countertop'] = countertop
data['microwave'] = microwave
data['electric'] = electric
data['convection'] = convection
data['gas'] = gas
data['compactor'] = compactor
data['desk'] = desk
data['dishwasher'] = dishwasher
data['sink'] = sink
data['freezer'] = freezer
data['garbage'] = garbage
data['pantry'] = pantry
data['grill'] = grill
data['lazy'] = lazy
data['refrigerator'] = refrigerator
data['purifier'] = purifier
data['warming'] = warming
data['wet'] = wet
data['wine'] = wine

data['appliance'] = data['appliance'].astype(int)
data['countertop'] = data['countertop'].astype(int)
data['microwave'] = data['microwave'].astype(int)
data['electric'] = data['electric'].astype(int)
data['convection'] = data['convection'].astype(int)
data['gas'] = data['gas'].astype(int)
data['compactor'] = data['compactor'].astype(int)
data['desk'] = data['desk'].astype(int)
data['dishwasher'] = data['dishwasher'].astype(int)
data['sink'] = data['sink'].astype(int)
data['freezer'] = data['freezer'].astype(int)
data['garbage'] = data['garbage'].astype(int)
data['pantry'] = data['pantry'].astype(int)
data['grill'] = data['grill'].astype(int)
data['lazy'] = data['lazy'].astype(int)
data['refrigerator'] = data['refrigerator'].astype(int)
data['purifier'] = data['purifier'].astype(int)
data['warming'] = data['warming'].astype(int)
data['wet'] = data['wet'].astype(int)
data['wine'] = data['wine'].astype(int)

In [23]:
feature = data["floor_covering"]
floor_chars = []
floor_vectors = []

for i in feature:
    description = i.lower().split(",")
    floor_chars += description

# Create a vector with all the unique values
labels = []

for i in floor_chars:
    if ':' in i:
        aux = i.split(':')[1] #Save the first keyword of the description after the ':'
        labels.append(aux) #Just keep the general feature
    #print(f"Saving {aux}")
    else:
        labels.append(i)
    #print(f"Saving {i}")

labels = np.unique(labels)

#These labels should be given to the programming team as additional data
labels = ['mexican','porcelain','stone','acrylic','wood','brick','carpet','ceramic',
          'concrete','vinyl','marble','other']

for i in feature:
    encoder = np.zeros(12)
    description = i.lower().split(",")

    for j in description:
        if ':' in j:
            j = j.split(':')[1]
    
    for k in range(len(labels)):
        if labels[k] == j or labels[k] in j:
            encoder[k] = 1
        else:
            if 'talavera' in j:
                encoder[0] = 1
            if 'italian' in j or 'porcelain wood' in j or 'porclain' in j:
                encoder[1] = 1
            if 'cantera' in j or 'canterra' in j:
                encoder[2] = 1
            if 'bamboo' in j or 'brazilian' in j:
                encoder[3] = 1
  
    floor_vectors.append(encoder)

# Create the columns with the presence of each feature
mexican = []
porcelain = []
stone = []
acrylic = []
wood = []
brick = []
carpet = []
ceramic = []
concrete = []
vinyl = []
marble = []
other = []

for i in floor_vectors:
    mexican.append(i[0])
    porcelain.append(i[1])
    stone.append(i[2])
    acrylic.append(i[3])
    wood.append(i[4])
    brick.append(i[5])
    carpet.append(i[6])
    ceramic.append(i[7])
    concrete.append(i[8])
    vinyl.append(i[9])
    marble.append(i[10])
    other.append(i[11])

data['mexican'] = mexican
data['porcelain'] = porcelain
data['stone'] = stone
data['acrylic'] = acrylic
data['wood'] = wood
data['brick'] = brick
data['carpet'] = carpet
data['ceramic'] = ceramic
data['concrete'] = concrete
data['vinyl'] = vinyl
data['marble'] = marble
data['other'] = other

data['mexican'] = data['mexican'].astype(int)
data['porcelain'] = data['porcelain'].astype(int)
data['stone'] = data['stone'].astype(int)
data['acrylic'] = data['acrylic'].astype(int)
data['wood'] = data['wood'].astype(int)
data['brick'] = data['brick'].astype(int)
data['carpet'] = data['carpet'].astype(int)
data['ceramic'] = data['ceramic'].astype(int)
data['concrete'] = data['concrete'].astype(int)
data['vinyl'] = data['vinyl'].astype(int)
data['marble'] = data['marble'].astype(int)
data['other'] = data['other'].astype(int)

In [24]:
data = data.drop(["kitchen_features", "floor_covering"], axis = 1).reset_index(drop = True)

### Dropping useless kitchen and material features

In [25]:
feats_drop = ["MLS", "taxes", "appliance", "countertop", "microwave", "gas", "compactor", "desk", "freezer",
             "garbage", "grill", "lazy", "wine", "mexican", "wood", "concrete", "marble", "other"]

In [26]:
data = data.drop(feats_drop, axis = 1).reset_index(drop = True)

In [27]:
train = data.sample(frac = 0.75, replace = True, random_state=43)
validate = data.sample(frac = 0.1, replace = True, random_state=44)
test = data.sample(frac = 0.15, replace = True, random_state=45)

### Scaling features

In [29]:
for d in [train,validate,test]:
    for c in d.columns.values:
        d[c] =(d[c] - d[c].mean()) / d[c].std()

In [32]:
train.to_csv("Train_Data.csv", index = False)
validate.to_csv("Validation.csv", index = False)
test.to_csv("Test_Data.csv", index = False)

In [581]:
data.to_csv("Scaled_Data.csv", index = False)

In [554]:
#data.to_csv("Unscaled_Data.csv", index = False)